# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.tail()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
284,284,saurimo,-9.6608,20.3916,62.82,24,0,4.81,AO,1723938557
285,285,labytnangi,66.6572,66.4183,55.26,84,65,3.33,RU,1723938558
286,286,sao jose da coroa grande,-8.8978,-35.1478,75.06,69,97,10.85,BR,1723938558
287,287,tutoia,-2.7619,-42.2744,77.76,80,5,10.29,BR,1723938558
288,288,vila franca do campo,37.7167,-25.4333,73.81,89,58,4.03,PT,1723938559


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
# %%capture --no-display

humidity_c = city_data_df.hvplot.points("Lng",
                                        "Lat",
                                        title = "Ideal Vacation Locations",
                                        geo = True,
                                        width = 800,
                                        height = 400,
                                        color = "City",
                                        size = "Humidity",
                                        tiles = "OSM",
                                        alpha = .75)

# Display the map
humidity_c

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# troubleshoot stats
print(city_data_df.describe())
print(city_data_df.head())

#test filter
city_data_df2 = city_data_df.loc[
    (city_data_df["Max Temp"] < 23) &
    (city_data_df["Max Temp"] > 14)
]

#Check stats
print(city_data_df2.head(5))
print(city_data_df.isnull().sum())
print(city_data_df.dtypes)

          City_ID         Lat         Lng    Max Temp    Humidity  Cloudiness  \
count  289.000000  289.000000  289.000000  289.000000  289.000000  289.000000   
mean   144.000000   18.050040   11.694282   68.310277   72.806228   54.027682   
std     83.571327   34.045146   93.502839   13.703422   19.651932   39.602354   
min      0.000000  -54.800000 -176.559700   24.930000    8.000000    0.000000   
25%     72.000000   -9.385200  -68.933300   58.150000   63.000000   12.000000   
50%    144.000000   19.640600   19.234500   69.910000   77.000000   63.000000   
75%    216.000000   45.523400   88.883300   78.780000   87.000000   97.000000   
max    288.000000   78.218600  177.483300  101.520000  100.000000  100.000000   

       Wind Speed          Date  
count  289.000000  2.890000e+02  
mean     7.899758  1.723938e+09  
std      5.872691  8.027990e+01  
min      0.000000  1.723938e+09  
25%      3.470000  1.723938e+09  
50%      6.420000  1.723939e+09  
75%     10.290000  1.723939e+09 

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df2 = city_data_df.loc[
    (city_data_df["Max Temp"] < 95) &
    (city_data_df["Max Temp"] > 50) &
    (city_data_df["Humidity"] < 50) &
    (city_data_df["Cloudiness"] < 30) &
    (city_data_df["Wind Speed"] < 10)
]

# Drop any rows with null values
city_data_df2 = city_data_df2.dropna()

# Display sample data
city_data_df2.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
78,78,nalut,30.3333,10.8500,87.73,22,0,7.85,LY,1723938483
87,87,ribas do rio pardo,-20.4431,-53.7592,78.60,22,0,6.20,BR,1723938487
97,97,rakops,-21.0167,24.3333,61.41,33,2,8.01,BW,1723938490
135,135,mwene-ditu,-7.0000,23.4500,71.11,33,3,2.42,CD,1723938504
142,142,turpan,42.9333,89.1667,94.87,15,23,1.48,CN,1723938506


In [30]:
city_data_df2['City_ID'].count()

21

### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df2[["City", 
                          "Country", 
                          "Lat", 
                          "Lng", 
                          "Humidity"]].copy()

In [37]:
hotel_df.head(5)

,City,Country,Lat,Lng,Humidity
78,nalut,LY,30.3333,10.8500,22
87,ribas do rio pardo,BR,-20.4431,-53.7592,22
97,rakops,BW,-21.0167,24.3333,33
135,mwene-ditu,CD,-7.0000,23.4500,33
142,turpan,CN,42.9333,89.1667,15


In [ ]:
# Add an empty column, "Hotel Name," to the DataFrame so yan store theou c hotel found using the Geoapify API
hotel_df.insert(5,"Hotel", "" ) #column, new column name, list

# Display sample data
hotel_df

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = {
    "apiKey": geoapify_key,
    "limit": limit,
    "categories": categories,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row ["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nalut - nearest hotel: No hotel found
ribas do rio pardo - nearest hotel: Hotel São Francisco
rakops - nearest hotel: Rakops River Lodge
mwene-ditu - nearest hotel: Hôtel KAMAS
turpan - nearest hotel: Oriental
san rafael - nearest hotel: Hotel Regional
solwezi - nearest hotel: Lodge 24
smithers - nearest hotel: Sunshine Inn Hotel
alenquer - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
alice springs - nearest hotel: Aurora Alice Springs
chardara - nearest hotel: Бақдаулет
sarpol-e zahab - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
coxim - nearest hotel: Avendita
saphane - nearest hotel: No hotel found
maun - nearest hotel: Center Lodge Conference Center
emerald - nearest hotel: Emerald Central Hotel
figuig (centre) - nearest hotel: No hotel found
the pas - nearest hotel: Wescana Inn
saurimo - nearest hotel: Hotel Princesinha


,City,Country,Lat,Lng,Humidity,Hotel Name
78,nalut,LY,30.3333,10.8500,22,No hotel found
87,ribas do rio pardo,BR,-20.4431,-53.7592,22,Hotel São Francisco
97,rakops,BW,-21.0167,24.3333,33,Rakops River Lodge
135,mwene-ditu,CD,-7.0000,23.4500,33,Hôtel KAMAS
142,turpan,CN,42.9333,89.1667,15,Oriental
154,san rafael,AR,-34.6177,-68.3301,48,Hotel Regional
161,solwezi,ZM,-12.1688,26.3894,46,Lodge 24
167,smithers,CA,54.7804,-127.1743,21,Sunshine Inn Hotel
171,alenquer,BR,-1.9417,-54.7383,48,No hotel found
182,al bawiti,EG,28.3492,28.8659,47,Old Oasis Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
# %%capture --no-display
# Configure the map plot
hotel_c = hotel_df.hvplot.points(
    "Lng", "Lat", title="Ideal Hotel Locations",
    geo=True, width=800, height=400,
    color="City", size="Humidity", 
    tiles="OSM", hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_c

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)